In [3]:
%pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 6.5 MB/s eta 0:00:00a 0:00:01m
  Attempting uninstall: cachetools━━━━━━━━━━━━━━━━━━━━━━━━━━━  4/13 [googleapis-common-protos]
    Found existing installation: cachetools 6.1.0━━━━━━━━━━━━━━━━━  5/13 [cachetools]on-protos]
    Uninstalling cachetools-6.1.0:━━━━━━━━━━━━━━━━━━━━━━━━  5/13 [cachetools]
      Successfully uninstalled cachetools-6.1.0━━━━━━━━━━━━━━━  5/13 [cachetools]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13/13 [langchain_google_genai]hain_google_genai]guage]
Note: you may need to restart the kernel to use updated packages.


API Keys

In [1]:
# set your API keys i nthe .env file..
import os
from dotenv import load_dotenv
load_dotenv()
langsmith_api_key=os.getenv('LANGCHAIN_API_KEY')
google_api_key=os.getenv('GOOGLE_API_KEY')

Tracability

In [5]:
from langchain.callbacks.tracers.langchain import LangChainTracer

# Enable tracing
tracer = LangChainTracer()

LLM

In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI

# create the LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-001",
    google_api_key=google_api_key,
    callbacks=[tracer])

System

In [7]:
# set the system prompt for the LLM here
from langchain_core.messages import SystemMessage
SystemMessage(content="You are a helpful assistant.")


SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={})

Knowledgebase

In [ ]:
# set up any DB's here

Tools

In [ ]:
# define any tools here

Orchestrator

In [ ]:
# set up LangChain/LangGraph here